In [1]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


In [2]:
price = pd.read_csv('./transpose-123 - transpose.csv')
supply = pd.read_excel('../data_2020.xlsx')
supply['년도'] = supply['거래년월일'].apply(lambda x: str(x)[:4]).astype(str)
supply['월'] = supply['거래년월일'].apply(lambda x: str(x)[4:6]).astype(str)
supply['일'] = supply['거래년월일'].apply(lambda x: str(x)[6:8]).astype(str)
supply['거래량'] = supply['거래량'].apply(lambda x: ''.join(str(x).split(','))).astype(int)
sup_total = supply.groupby(['년도','월','일'], as_index=False).sum()
sup_total = sup_total[:-2]
sup_total.head()

,년도,월,일,시장코드,거래량,품목코드
0,2020,01,02,371501.0,6,120701.0
1,2020,01,03,690014.0,1286,362103.0
2,2020,01,04,1071509.0,134,482804.0
3,2020,01,06,371501.0,9,120701.0
4,2020,01,07,230003.0,12,120701.0


In [3]:
price['년도'] = price['년도'].apply(lambda x: str(x))
price['월'] = price['월/일'].apply(lambda x: str(x)[0:2]).astype(str)
price['일'] = price['월/일'].apply(lambda x: str(x)[3:5]).astype(str)
price.head()

,년도,월/일,평균,월,일
0,2000,01/03,"103,000",01,03
1,2000,01/04,"97,000",01,04
2,2000,01/05,"95,000",01,05
3,2000,01/06,"98,000",01,06
4,2000,01/07,"98,000",01,07


In [4]:
df = pd.merge(price, sup_total, on=['년도','월','일'])
df.head()

,년도,월/일,평균,월,일,시장코드,거래량,품목코드
0,2020,01/02,"63,400",01,02,371501.0,6,120701.0
1,2020,01/03,"60,000",01,03,690014.0,1286,362103.0
2,2020,01/06,"58,800",01,06,371501.0,9,120701.0
3,2020,01/07,"56,200",01,07,230003.0,12,120701.0
4,2020,01/09,"51,200",01,09,601502.0,18,241402.0


In [5]:
df2 = df[['년도','월','일','평균','거래량']]
df2['년도'] = df2['년도'].apply(lambda x: int(x))
df2['월'] = df2['월'].apply(lambda x: int(x))
df2['일'] = df2['일'].apply(lambda x: int(x))
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 

df2

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,년도,월,일,평균,거래량
0,2020,1,2,63400,6
1,2020,1,3,60000,1286
2,2020,1,6,58800,9
3,2020,1,7,56200,12
4,2020,1,9,51200,18
5,2020,1,10,51200,496
6,2020,1,13,56200,912
7,2020,1,14,60800,457
8,2020,1,15,63600,936
9,2020,1,16,61800,705


In [6]:
a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a

del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)
df2.head(10)

y = df2['d+1']
del df2['d+1']

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df2.head(10)

,년도,월,일,거래량,d-0,d-1,d-2,d-3,d-4,d-5,d-6
0,2020,1,15,936,60800,56200,51200,51200,56200,58800,60000
1,2020,1,16,705,63600,60800,56200,51200,51200,56200,58800
2,2020,1,17,62,61800,63600,60800,56200,51200,51200,56200
3,2020,1,20,270,62400,61800,63600,60800,56200,51200,51200
4,2020,1,21,555,68200,62400,61800,63600,60800,56200,51200
5,2020,1,22,386,77400,68200,62400,61800,63600,60800,56200
6,2020,1,23,65,78800,77400,68200,62400,61800,63600,60800
7,2020,1,30,243,77200,78800,77400,68200,62400,61800,63600
8,2020,1,31,253,69400,77200,78800,77400,68200,62400,61800
9,2020,2,3,150,65600,69400,77200,78800,77400,68200,62400


In [8]:
seed_pred_result = []
rmse_result = []
for i in range(10):
    rand_num = i
    x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.9, random_state=rand_num)
    print(y_test.shape)
    models = [{'name':'XGBoost'},
              {'name': 'GradientBoosting'},
              {'name':'LightGBM'},
              {'name': 'Lasso'}]

    result = []
    rmse = []
    result.append(y_test)

    for m in models:
        loaded_model = pickle.load(open("pima_{}_{}.pickle.dat".format(i, m['name']), "rb"))
        predictions = loaded_model.predict(x_test.values) 
        result.append(predictions)
        rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})
    
    
    
    
    model = keras.models.load_model('./keras_{}.hdf5'.format(i))
    
    modelPred = model.predict(x_test.values)
    Y_prediction = modelPred.flatten()

    result.append(Y_prediction)
    rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

    dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

    df_res = pd.DataFrame(dict)
    df_res['average'] = df_res[['xgboost', 'gboost', 'lightgbm', 'lasso', 'keras']].apply(lambda x: round(x.mean()), axis=1)
    rmse.append({'average':sqrt(mean_squared_error(y_test, df_res['average']))})

    rmse_result.append(rmse)
    seed_pred_result.append(df_res)    

(79,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)
(79,)


In [9]:
rmse_result

[[{'XGBoost': 8134.2743206850155},
  {'GradientBoosting': 9126.844284900004},
  {'LightGBM': 7685.973072086535},
  {'Lasso': 7447.332893576969},
  {'keras': 7575.346815152439},
  {'average': 7728.374830388815}],
 [{'XGBoost': 8600.158992409728},
  {'GradientBoosting': 8743.228697878658},
  {'LightGBM': 8022.490872462624},
  {'Lasso': 7472.445995953337},
  {'keras': 7605.271989598875},
  {'average': 7839.32234703737}],
 [{'XGBoost': 7646.224821496096},
  {'GradientBoosting': 8282.3237933941},
  {'LightGBM': 7741.177801869259},
  {'Lasso': 7488.280875074549},
  {'keras': 7640.645128591945},
  {'average': 7529.284581980927}],
 [{'XGBoost': 8573.444296340953},
  {'GradientBoosting': 8242.322409317221},
  {'LightGBM': 7644.152904156703},
  {'Lasso': 7528.4891738276465},
  {'keras': 7660.416772290252},
  {'average': 7661.425431704574}],
 [{'XGBoost': 7600.383391813522},
  {'GradientBoosting': 8461.488041126791},
  {'LightGBM': 7361.668007204304},
  {'Lasso': 7344.749178102676},
  {'keras': 7

In [10]:
seed_pred_result[0].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
2,62400,65551.398438,63589.507451,63263.739120,62405.117164,62859.894531,63534.0
13,49600,49971.234375,49046.410095,47311.939805,46707.531291,46353.671875,47878.0
53,147600,101752.070312,102120.402741,105369.552267,98723.396753,98088.585938,101211.0
41,121400,116254.460938,118694.435885,121666.858147,120532.407283,119935.500000,119417.0
66,64600,63538.328125,62416.621178,62732.009367,63412.893835,62950.601562,63010.0


In [11]:
seed_pred_result[1].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
38,131200,137720.718750,141956.973248,139698.998503,132932.813579,132258.265625,136914.0
44,123600,126841.601562,124545.811448,125435.653572,122314.894640,121847.570312,124197.0
27,73750,75928.414062,74785.635901,75491.428730,74706.294755,74634.414062,75109.0
40,121000,113157.929688,120321.385913,114564.269449,118731.897936,117988.789062,116953.0
36,128000,122560.617188,124296.205041,120613.919781,122948.199880,122813.203125,122646.0


In [12]:
seed_pred_result[2].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
44,123600,122348.000000,121814.795863,118703.568133,122657.051157,120466.960938,121198.0
35,120000,107461.257812,110607.450253,111487.755472,114281.948019,113343.953125,111436.0
82,42000,39187.855469,38845.167403,39045.629096,39859.311521,39484.273438,39284.0
0,63600,62410.535156,61190.660613,62129.164568,62334.263061,61725.644531,61958.0
11,47800,52310.054688,53518.968826,49600.620583,51168.963247,49759.476562,51272.0


In [13]:
seed_pred_result[3].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
62,76200,85940.843750,79918.312344,82771.086272,78631.841103,77851.164062,81023.0
75,42000,42024.847656,41496.569220,42728.153542,41002.811327,40931.093750,41637.0
9,55400,65977.468750,66986.897392,65948.269025,65517.478242,64999.445312,65886.0
45,123600,114246.914062,122474.526795,120368.109776,121973.954387,121468.210938,120106.0
34,112000,111845.796875,104988.892001,108953.938421,110943.147321,111663.453125,109679.0


In [14]:
seed_pred_result[4].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
76,40000,42936.644531,41968.969224,42681.384773,42810.596313,43069.898438,42693.0
83,45600,42269.421875,41968.969224,43054.694757,43518.339000,43874.140625,42937.0
14,55200,50422.878906,50238.000833,49743.567263,50174.576244,50548.609375,50226.0
35,120000,114045.968750,111055.146786,111592.834454,114314.266957,114619.687500,113126.0
27,73750,73031.117188,74383.325891,74713.358712,74677.231756,74783.875000,74318.0
